In [1]:
import re, os, signal

import numpy as np
import pandas as pd
import modanez_tools as mt

from pathlib import Path
from datetime import datetime

In [2]:
# Carregar o dataframe df_acordaos
start_time = datetime.now()

df_acordaos = pd.read_csv('dados/acordaos.csv', sep=',', error_bad_lines=False, warn_bad_lines=False, low_memory=False)
df_acordaos.rename(str.upper, axis='columns', inplace=True)

df_acordaos.insert(0, 'ID', range(1, len(df_acordaos)+1))
df_acordaos.ID = df_acordaos.ID.map(str)
df_acordaos.ANOACORDAO = df_acordaos.ANOACORDAO.map(str)
df_acordaos.insert(13, 'TIPO_DECISAO', np.NaN)

print('Duration: {}'.format(datetime.now() - start_time))
df_acordaos.info()

Duration: 0:00:38.898555
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309806 entries, 0 to 309805
Data columns (total 30 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   ID                                309806 non-null  object 
 1   ACORDAO                           288120 non-null  object 
 2   ADVOGADO                          223477 non-null  object 
 3   ANOACORDAO                        309806 non-null  object 
 4   ASSUNTO                           77524 non-null   object 
 5   COLEGIADO                         309806 non-null  object 
 6   DATASESSAO                        308360 non-null  object 
 7   DECISAO                           19451 non-null   object 
 8   DECLARACAOVOTO                    3328 non-null    object 
 9   ENTIDADE                          253844 non-null  object 
 10  INTERESSADOS                      255674 non-null  object 
 11  MINISTROALEGOUIMPEDIMENTOSE

In [3]:
df_acordaos.drop(['REPRESENTANTEMP', 'UNIDADETECNICA', 'UNIDADETECNICA', 'MINISTROREVISOR',
                  'MINISTROAUTORVOTOVENCEDOR', 'MINISTROAPRESENTOUDECLARACAOVOTO',
                 'MINISTROALEGOUIMPEDIMENTOSESSAO', 'INTERESSADOS', 'RECURSOS', 'ENTIDADE',
                 'DECLARACAOVOTO', 'ADVOGADO', 'RELATORDELIBERACAORECORRIDA'], axis=1, inplace=True)

df_acordaos.rename(columns={'ANOACORDAO':'ANO_ACORDAO', 'NUMACORDAO':'NUMERO_ACORDAO',
                           'TIPOPROCESSO':'TIPO_PROCESSO', 'DATASESSAO':'DATA_SESSAO',
                           'NUMATA':'NUMERO_ATA', 'PROC':'PROCESSO'}, inplace=True)

df_acordaos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309806 entries, 0 to 309805
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   309806 non-null  object 
 1   ACORDAO              288120 non-null  object 
 2   ANO_ACORDAO          309806 non-null  object 
 3   ASSUNTO              77524 non-null   object 
 4   COLEGIADO            309806 non-null  object 
 5   DATA_SESSAO          308360 non-null  object 
 6   DECISAO              19451 non-null   object 
 7   TIPO_DECISAO         0 non-null       float64
 8   NUMERO_ACORDAO       309806 non-null  object 
 9   NUMERO_ATA           307239 non-null  object 
 10  PROCESSO             307121 non-null  object 
 11  RELATOR              307262 non-null  object 
 12  RELATORIO            110737 non-null  object 
 13  SUMARIO              115466 non-null  object 
 14  TIPO_PROCESSO        298928 non-null  object 
 15  VOTO             

In [4]:
# DE -> DECISAO = 19451 : 647 marcados como SIGILOSO por esse motivo não possui ACORDAO, VOTO, RELATORIO, SUMARIO
df_acordaos.loc[(df_acordaos.DECISAO.notnull() == True) & ((df_acordaos.ACORDAO.isnull()==True) | (df_acordaos.ACORDAO.str.contains('SIGILOSO', na=False))), 'TIPO_DECISAO'] = 'DE'

In [5]:
# AR -> ACORDAO DE RELACAO : 193963
df_acordaos.loc[(df_acordaos.ACORDAO.notnull()) & (df_acordaos.DECISAO.isnull()) & (df_acordaos.RELATORIO.isnull()) & (df_acordaos.VOTO.isnull()), 'TIPO_DECISAO'] = 'AR'

In [6]:
df_acordaos.loc[df_acordaos.TIPO_DECISAO.isna(), 'TIPO_DECISAO'] = 'AC'

In [7]:
df_acordaos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309806 entries, 0 to 309805
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ID                   309806 non-null  object
 1   ACORDAO              288120 non-null  object
 2   ANO_ACORDAO          309806 non-null  object
 3   ASSUNTO              77524 non-null   object
 4   COLEGIADO            309806 non-null  object
 5   DATA_SESSAO          308360 non-null  object
 6   DECISAO              19451 non-null   object
 7   TIPO_DECISAO         309806 non-null  object
 8   NUMERO_ACORDAO       309806 non-null  object
 9   NUMERO_ATA           307239 non-null  object
 10  PROCESSO             307121 non-null  object
 11  RELATOR              307262 non-null  object
 12  RELATORIO            110737 non-null  object
 13  SUMARIO              115466 non-null  object
 14  TIPO_PROCESSO        298928 non-null  object
 15  VOTO                 110475 non-nu

In [8]:
df_acordaos.NUMERO_ACORDAO = df_acordaos.NUMERO_ACORDAO.apply(mt.format_numero_acordao)

In [9]:
start_time = datetime.now()

df_acordaos.ID = df_acordaos.apply(mt.criaChaveAcordao, axis=1)

print('Duration: {}'.format(datetime.now() - start_time))

Duration: 0:00:07.378633


In [10]:
# 2871 registros com os campos, ACORDAO, DECISAO, VOTO e RELATORIO nulos simultaneamente e por isso excluídos
df_acordaos.drop(df_acordaos.loc[(df_acordaos.ACORDAO.isnull()) & df_acordaos.DECISAO.isnull() & df_acordaos.RELATORIO.isnull() & df_acordaos.VOTO.isnull()].index, inplace=True)

In [11]:
# Registros marcados como SILIGOSO e por esse motivo convertidos para NaN a fim de padronização
df_acordaos.loc[(df_acordaos.ACORDAO.str.lower() == 'sigiloso'), 'ACORDAO'] = np.NaN
df_acordaos.loc[(df_acordaos.DECISAO.str.lower() == 'sigiloso'), 'DECISAO'] = np.NaN
df_acordaos.loc[(df_acordaos.RELATORIO.str.lower() == 'sigiloso'), 'RELATORIO'] = np.NaN
df_acordaos.loc[(df_acordaos.VOTO.str.lower() == 'sigiloso'), 'VOTO'] = np.NaN

In [12]:
df_acordaos.fillna('', inplace=True)

In [13]:
df_acordaos['DECISAO'] = df_acordaos.ACORDAO + ' ' + df_acordaos.DECISAO + ' ' + df_acordaos.RELATORIO + ' ' + df_acordaos.VOTO + ' ' + df_acordaos.VOTOCOMPLEMENTAR + ' ' + df_acordaos.VOTOMINISTROREVISOR

In [14]:
# Retira colunas que não interessam para o trabalho
df_acordaos = df_acordaos.drop(['ANO_ACORDAO', 'ASSUNTO', 'COLEGIADO', 'DATA_SESSAO', 'NUMERO_ACORDAO', 
                                'NUMERO_ATA', 'PROCESSO', 'RELATOR', 'SUMARIO', 'TIPO_PROCESSO', 'ACORDAO', 
                                'VOTO', 'RELATORIO', 'VOTOCOMPLEMENTAR', 'VOTOMINISTROREVISOR', 
                                'TIPO_DECISAO'], axis=1)

df_acordaos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306935 entries, 0 to 309804
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   ID       306935 non-null  object
 1   DECISAO  306935 non-null  object
dtypes: object(2)
memory usage: 7.0+ MB


In [15]:
df_acordaos.sample(2)

,ID,DECISAO
301355,AC-003468-2018-2C,"<p>\tVISTOS, relatados e discutidos estes auto..."
281945,AC-004217-2017-2C,"<p>VISTOS, relatados e discutidos estes autos ..."


In [16]:
df_acordaos.tail(5)

,ID,DECISAO
309800,AC-000058-2018-PL,"<p class=""tCU_-_Ac_-_item_9_-_§§_1"">Vistos, re..."
309801,AC-000031-2018-PL,"<p class=""normal_1"">VISTOS, relatados e discut..."
309802,AC-000033-2018-PL,"<p class=""normal_1"">VISTOS, relatados e discut..."
309803,AC-000030-2018-PL,"<p class=""normal_1"">VISTOS, relatados e discut..."
309804,AR-000061-2018-PL,Documento classificado como sigiloso com funda...


In [17]:
for path in Path('dados/').rglob('**/*.txt'):
    texto = ''
    with path.open() as f:
        texto += f.read()   
    if df_acordaos.ID.str.contains(path.stem).any() == False :
        df_acordaos = df_acordaos.append({'ID': path.stem, 'DECISAO': texto}, ignore_index=True)

In [18]:
df_acordaos.tail(5)

,ID,DECISAO
306980,AC-002740-2018-PL,"Relatório\nAdoto, como Relatório, a instrução ..."
306981,AC-002939-2018-PL,Relatório\nTrata-se de representação apresenta...
306982,AC-002743-2018-PL,Relatório\nTrata-se de apartado da auditoria d...
306983,AR-002346-2018-PL,Acórdão\nACÓRDÃO Nº 2346/2018 - TCU - Plenário...
306984,AC-012879-2018-1C,"Relatório\nAdoto, como relatório, a instrução ..."


In [ ]:
mt.save_file(df_acordaos, 'dados/df_acordaos.csv')

Duration: 0:01:09.769327


In [ ]:
os.kill(os.getpid(), signal.SIGKILL)